In [14]:
%pip install --upgrade jupyter ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ----------------- ---------------------- 61.4/139.4 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ----------------------------------- --- 194.6/215.0 kB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 215.0/215.0 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 7.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.3 MB 7.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.9/2.3 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/257.4 kB ? eta -:--:--
   -----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [18]:
df.columns

Index(['client_id', 'npo_account_id', 'npo_accnts_nmbr', 'pmnts_type', 'year',
       'quarter', 'gender', 'age', 'clnt_cprtn_time_d', 'actv_prd_d',
       'lst_pmnt_rcnc_d', 'balance', 'oprtn_sum_per_qrtr',
       'oprtn_sum_per_year', 'frst_pmnt_date', 'lst_pmnt_date_per_qrtr',
       'frst_pmnt', 'lst_pmnt', 'pmnts_sum', 'pmnts_nmbr',
       'pmnts_sum_per_qrtr', 'pmnts_sum_per_year', 'pmnts_nmbr_per_qrtr',
       'pmnts_nmbr_per_year', 'incm_sum', 'incm_per_qrtr', 'incm_per_year',
       'mgd_accum_period', 'mgd_payment_period', 'lk', 'assignee_npo',
       'assignee_ops', 'region', 'citizen', 'fact_addrss', 'appl_mrkr',
       'evry_qrtr_pmnt', 'churn', 'id_', 'lst_pmnt_date_per_qrtr_day',
       'lst_pmnt_date_per_qrtr_month', 'lst_pmnt_date_per_qrtr_year',
       'day_sum1', 'quarter1', 'balance_quarter', 'oprtn_sum_per_qrtr_quarter',
       'oprtn_sum_per_year_quarter', 'pmnts_sum_quarter',
       'incm_per_year_quarter', 'pmnts_nmbr_quarter',
       'pmnts_sum_per_qrtr_quarter

In [21]:

import numpy as np
import gc 
import pandas as pd
import optuna
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt

import random
random.seed(42)
np.random.seed(42)

ImportError: cannot import name 'plot_roc_curve' from 'sklearn.metrics' (c:\Users\Andrey\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\__init__.py)

In [2]:
target_col = "churn"

In [3]:
def get_features(df):
    df = df.drop(['slctn_nmbr','postal_code', 'phone_number', 'email'], axis=1)
    df['region'] = LabelEncoder().fit_transform(df["region"])
    df = df.drop_duplicates()
    df['id_'] = df['quarter'] + df['npo_account_id']
    df = df.drop_duplicates("id_")
    
    df['lst_pmnt_date_per_qrtr'] = pd.to_datetime(df['lst_pmnt_date_per_qrtr'])
    df['lst_pmnt_date_per_qrtr_day'] = df['lst_pmnt_date_per_qrtr'].dt.day
    df['lst_pmnt_date_per_qrtr_month'] = df['lst_pmnt_date_per_qrtr'].dt.month
    df['lst_pmnt_date_per_qrtr_year'] = df['lst_pmnt_date_per_qrtr'].dt.year
    df['day_sum1'] = df['lst_pmnt_date_per_qrtr_month']*30+df['lst_pmnt_date_per_qrtr_day']
    df['quarter1'] = LabelEncoder().fit_transform(df["quarter"].copy())
    
    col = ['balance', 'oprtn_sum_per_qrtr', 'oprtn_sum_per_year', 'pmnts_sum', 'incm_per_year',
    'pmnts_nmbr', 'pmnts_sum_per_qrtr', 'pmnts_sum_per_year', 'pmnts_nmbr_per_qrtr', 'pmnts_nmbr_per_year', 'incm_sum', 'incm_per_qrtr']

    for c in tqdm(col):
        data1 = df.groupby('npo_account_id')[c].mean().reset_index()
        data1.columns = ['npo_account_id', f'{c}_quarter']
        df = pd.merge(df, data1, on='npo_account_id', how='left')
        
    data1 = df.groupby('npo_account_id')["quarter"].nunique().reset_index()
    data1.columns = ['npo_account_id', f'quarter_size']
    df = pd.merge(df, data1, on='npo_account_id', how='left')
    df['quartal'] = df['quarter'].str[-1].astype('int32')
    return df

In [4]:
df = pd.read_parquet("C:/Users/Andrey/Documents/Ml/train.parquet")
df = get_features(df)

100%|██████████| 12/12 [00:45<00:00,  3.78s/it]


In [5]:
X = df.drop(columns=[target_col, 'id_', 'client_id', 'npo_account_id', 'quarter',
    'lst_pmnt_date_per_qrtr', 'frst_pmnt_date', 'quarter']).values 
y = df[target_col].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
'''def objective(trial):
    params = {
        'device':'gpu',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': 1,
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 6, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-4, 1),
        'n_estimators': 50,
        'num_leaves ': trial.suggest_int('num_leaves', 3, 100)
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)
print(study.best_params)
print(study.best_value)
'''

In [17]:
score_list_acc = []
score_list_auc = []
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, valid_index in strat_kfold.split(X, y): 
    X_train, X_val = X[train_index], X[valid_index]
    y_train, y_val = y[train_index], y[valid_index]
    
    # 1500
    lgb_params = {'max_depth': 18, 'min_child_samples': 96, 'min_child_weight': 0.13310829983031214, 'num_leaves': 85,
                  'n_estimators':2350, 'device':'gpu', 'boosting_type': 'gbdt'}
    lgb = LGBMClassifier(**lgb_params)
    cat = CatBoostClassifier(iterations=5000, verbose=100, random_state=42, task_type="GPU", devices='0')#3000
    model = VotingClassifier(estimators=[('catboost', cat), ('lgbm', lgb)], voting='soft')
    model.fit(X_train, y_train)
    predict = model.predict_proba(X_val)[:, 1]
    
    score_list_auc.append(roc_auc_score(y_val, predict))
    score_list_acc.append(accuracy_score(y_val, np.round(predict)))
    
    print('roc_auc_score', roc_auc_score(y_val, predict))
    print('accuracy_score', accuracy_score(y_val, np.round(predict)))
    print()
    break
    
print('roc_auc_score', np.mean(score_list_auc))
print('accuracy_score', np.mean(score_list_acc))

# iter = 1200
#roc_auc_score 0.9967203256838154
#accuracy_score 0.9953078990898689

# iter = 1500
#roc_auc_score 0.9969534858449924
#accuracy_score 0.9954274661400886

#roc_auc_score 0.9984288345924386
#accuracy_score 0.9963405236144924

#quartal+
#roc_auc_score 0.9984746575073541
#accuracy_score 0.9963795431646806

#roc_auc_score 0.9984746575073541
#accuracy_score 0.9963795431646806

#ensemble+
#roc_auc_score 0.9981023210843869
#accuracy_score 0.9967850677755651

#roc_auc_score 0.9981023210843869
#accuracy_score 0.9967850677755651

Learning rate set to 0.005144
0:	learn: 0.6792426	total: 63.5ms	remaining: 5m 17s
100:	learn: 0.1093245	total: 6.09s	remaining: 4m 55s
200:	learn: 0.0499234	total: 11.7s	remaining: 4m 40s
300:	learn: 0.0370382	total: 17.6s	remaining: 4m 33s
400:	learn: 0.0322372	total: 23.4s	remaining: 4m 28s
500:	learn: 0.0296422	total: 29.2s	remaining: 4m 22s
600:	learn: 0.0279792	total: 35s	remaining: 4m 15s
700:	learn: 0.0268258	total: 40.5s	remaining: 4m 8s
800:	learn: 0.0257960	total: 46.1s	remaining: 4m 1s
900:	learn: 0.0248215	total: 51.9s	remaining: 3m 56s
1000:	learn: 0.0240693	total: 57.5s	remaining: 3m 49s
1100:	learn: 0.0234044	total: 1m 3s	remaining: 3m 43s
1200:	learn: 0.0228247	total: 1m 8s	remaining: 3m 37s
1300:	learn: 0.0222773	total: 1m 14s	remaining: 3m 31s
1400:	learn: 0.0217867	total: 1m 19s	remaining: 3m 25s
1500:	learn: 0.0213496	total: 1m 25s	remaining: 3m 19s
1600:	learn: 0.0209247	total: 1m 31s	remaining: 3m 13s
1700:	learn: 0.0205534	total: 1m 36s	remaining: 3m 7s
1800:	lea

In [20]:

plot_roc_curve(model, X_test, y_test)

NameError: name 'plot_roc_curve' is not defined

In [32]:
test_df = pd.read_csv("test.csv")

In [33]:
test_df = get_features(test_df)
test_x =test_df.drop(columns=['id_', 'client_id', 'npo_account_id', 'quarter',
    'lst_pmnt_date_per_qrtr', 'frst_pmnt_date', 'quarter']).values 

100%|██████████| 12/12 [00:10<00:00,  1.17it/s]


In [37]:
predict = model.predict(test_x)
predidct_dataframe = test_df[["npo_account_id", "quarter"]]
predidct_dataframe['predict'] = round(predict)
predidct_dataframe.to_csv("predict.csv")

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
def predict_one_npo_account_id(test_data, npo_account_id, impo=False):
    """
    Возращает для всех определенного  пользователя вероятности
     impo возращать веротяности или False=нет
    """
    col = ['npo_accnts_nmbr', 'pmnts_type', 'year', 'gender', 'age',
       'clnt_cprtn_time_d', 'actv_prd_d', 'lst_pmnt_rcnc_d', 'balance',
       'oprtn_sum_per_qrtr', 'oprtn_sum_per_year', 'frst_pmnt', 'lst_pmnt',
       'pmnts_sum', 'pmnts_nmbr', 'pmnts_sum_per_qrtr', 'pmnts_sum_per_year',
       'pmnts_nmbr_per_qrtr', 'pmnts_nmbr_per_year', 'incm_sum',
       'incm_per_qrtr', 'incm_per_year', 'mgd_accum_period',
       'mgd_payment_period', 'lk', 'assignee_npo', 'assignee_ops', 'region',
       'citizen', 'fact_addrss', 'appl_mrkr', 'evry_qrtr_pmnt',
       'lst_pmnt_date_per_qrtr_day', 'lst_pmnt_date_per_qrtr_month',
       'lst_pmnt_date_per_qrtr_year', 'day_sum1', 'quarter1',
       'balance_quarter', 'oprtn_sum_per_qrtr_quarter',
       'oprtn_sum_per_year_quarter', 'pmnts_sum_quarter',
       'incm_per_year_quarter', 'pmnts_nmbr_quarter',
       'pmnts_sum_per_qrtr_quarter', 'pmnts_sum_per_year_quarter',
       'pmnts_nmbr_per_qrtr_quarter', 'pmnts_nmbr_per_year_quarter',
       'incm_sum_quarter', 'incm_per_qrtr_quarter', 'quarter_size']
    
    test_data = test_data.loc[(test_data.npo_account_id==npo_account_id)]
    
    if impo:
        predict = model.predict_proba(test_data[col].values)[:, 1]
    else:
        predict = model.predict(test_data[col].values)
        
    predidct_dataframe = test_data[["npo_account_id", "quarter"]]
    predidct_dataframe['churn'] = predict
    return predidct_dataframe

In [ ]:
predidct_dataframe.loc[predidct_dataframe.npo_account_id=="0x9DADF88CB3407C4E89403315F640393E"]

In [ ]:
import seaborn as sns
sns.lineplot(predidct_dataframe.loc[predidct_dataframe.npo_account_id=="0x9DADF88CB3407C4E89403315F640393E"].predict)